In [54]:
# from dotenv import load_dotenv

# _ = load_dotenv()


In [81]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from dotenv import load_dotenv
from contextlib import ExitStack

In [82]:
load_dotenv(override = True)
Tavily_api_key=os.environ.get("TAVILY_API_KEY")
openai_api_key=os.environ.get("OPENAI_API_KEY")


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


In [83]:
tool = TavilySearchResults(max_results=2)

In [84]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [85]:
from langgraph.checkpoint.sqlite import SqliteSaver
stack = ExitStack()
memory = stack.enter_context(SqliteSaver.from_conn_string(":memory:"))

In [86]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [87]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")


In [88]:
abot = Agent(model, [tool], system=prompt, checkpointer=memory)
messages = [HumanMessage(content="What is the weather in sf?")]

In [89]:
thread = {"configurable": {"thread_id": "1"}}

In [90]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
# result = abot.graph.invoke({"messages": messages})
# print(result['messages'][-1].content)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MMiGSW8WDGH5B3PYSOt1KpDx', 'function': {'arguments': '{"query":"San Francisco weather forecast"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 151, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1eeef225-0678-43c0-88c9-d4bfa40fc574-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather forecast'}, 'id': 'call_MMiGSW8WDGH5B3PYSOt1KpDx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 22, 'total_tokens': 173, 'input_token

In [91]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_se9rR9k6dwuGBbNzcvX2Lz08', 'function': {'arguments': '{"query":"Los Angeles weather forecast"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 891, 'total_tokens': 913, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ffb42ef6-c184-45c5-8093-d602014738ba-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles weather forecast'}, 'id': 'call_se9rR9k6dwuGBbNzcvX2Lz08', 'type': 'tool_call'}], usage_metadata={'input_tokens': 891, 'output_tokens': 22, 'total_tokens': 913, 'input_token_det

In [92]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Los Angeles is currently warmer than San Francisco. Los Angeles has a temperature of 55.0°F (12.8°C), while San Francisco has a temperature of 48.0°F (8.9°C).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 1796, 'total_tokens': 1842, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1664}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-5717ef68-0c55-4495-b3a3-66d5c77cfb27-0', usage_metadata={'input_tokens': 1796, 'output_tokens': 46, 'total_tokens': 1842, 'input_token_details': {'audio': 0, 'cache_read': 1664}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [93]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="Could you please clarify what you're comparing? Are you asking about the temperature between two specific locations, times of year, or something else entirely? Let me know so I can assist you further!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 149, 'total_tokens': 189, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-f4531ae2-c989-4ccb-8825-41418314931d-0', usage_metadata={'input_tokens': 149, 'output_tokens': 40, 'total_tokens': 189, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}
